In [5]:
import pandas 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer   #type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences  #type: ignore
import os

path = 'C:/Users/Khalid Almassri/Desktop/IMDB Dataset.csv'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        print(file_path)


In [6]:
data = pandas.read_csv(path)
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
data.shape

(50000, 2)

In [8]:
data.replace({'sentiment':{'positive':1, 'negative':0}}, inplace=True)
data.head()

C:\Users\Khalid Almassri\AppData\Local\Temp\ipykernel_33276\1626632432.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment':{'positive':1, 'negative':0}}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [9]:
training_data, testing_data = train_test_split(data, test_size=0.25, random_state=32)

In [10]:
print(training_data.shape)
print(testing_data.shape)

(37500, 2)
(12500, 2)


In [11]:
tokenizer=Tokenizer(num_words=4000, oov_token="<OOV>")
tokenizer.fit_on_texts(training_data["review"])
x_train = pad_sequences(tokenizer.texts_to_sequences(training_data['review']), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(testing_data['review']), maxlen=200)

In [12]:
l_train=training_data["sentiment"]
l_test=testing_data["sentiment"]

In [13]:
model= Sequential()
model.add(Embedding(input_dim=4000, output_dim=128))
model.add(LSTM(128, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(1, activation="sigmoid"))

In [14]:
model.compile(optimizer="adam",loss="binary_crossentropy", metrics=['accuracy'])

In [15]:
model.fit(x_train, l_train, epochs=5, batch_size=64, validation_split=0.25)

Epoch 1/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 69s 153ms/step - accuracy: 0.7046 - loss: 0.5459 - val_accuracy: 0.8293 - val_loss: 0.4015
Epoch 2/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 71s 162ms/step - accuracy: 0.8230 - loss: 0.3998 - val_accuracy: 0.8411 - val_loss: 0.3782
Epoch 3/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 63s 143ms/step - accuracy: 0.8099 - loss: 0.4186 - val_accuracy: 0.8286 - val_loss: 0.3865
Epoch 4/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 64s 145ms/step - accuracy: 0.8569 - loss: 0.3387 - val_accuracy: 0.8509 - val_loss: 0.3577
Epoch 5/5
440/440 ━━━━━━━━━━━━━━━━━━━━ 69s 158ms/step - accuracy: 0.8792 - loss: 0.3023 - val_accuracy: 0.8658 - val_loss: 0.3208


In [20]:
loss, accuracy = model.evaluate(x_test,l_test)
print(f'test loss: {loss}')
print(f'test accuracy: {accuracy}')

391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.8657 - loss: 0.3308
test loss: 0.3269621729850769
test accuracy: 0.8659200072288513


In [39]:
def sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=250)
    prediction = model.predict(padded_sequence)
    if prediction[0][0] > 0.5:
        sentiments = "positive"
    else:
        sentiments = "negative"
    return sentiments

In [40]:
r="this movie was amazing, i did enjoy the actors and everything"
sen = sentiment(r)
print(sen)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
positive
